In [2]:
!pip install -r requirements.txt
#!pip install fastapi
#pip install --upgrade pip
#print("Hello world")

     -------------------------------------- 453.6/453.6 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 207.2/207.2 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 4.2 MB/s eta 0:00:00
     -------------------------------------- 246.5/246.5 kB 3.8 MB/s eta 0:00:00
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
     ---------------------------------------- 2.1/2.1 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 67.0/67.0 kB 3.5 MB/s eta 0:00:00


In [3]:
import selenium.common.exceptions
import urllib3.exceptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from time import sleep
import threading
import csv
import traceback
import multiprocessing
from typing import Union
from fastapi import FastAPI
import json

In [4]:
def split_list(lst, k):
    """
    This function is used to split a list to sub-lists that have k items.

    Args:
        lst (list): input list
        k (int): number of item of a sub-list

    Returns:
        list: list of sub-list
    For example:
    split_list([1,2,3,4,5],2) => [[1,2],[3,4],[5]]
    """
    size = len(lst)
    return [lst[i:i+k] for i in range(0, size, k)]


print(split_list(["hi", 2, 3, 4, 5], 3))
print([1,2,3,4,5,6][:-1])

[['hi', 2, 3], [4, 5]]
[1, 2, 3, 4, 5]


In [5]:
def get_first_not_null_item(lsts):
    """
      this function is used to get the first not null item of a list, it there is no not-null item,
      this func returns the last item
      
    Args:
        lsts (list): input list

    Returns:
        string: the first not null item
    """
    for l in lsts:
        if (len(l)!=0):
            return l
    return lsts[-1]
print(get_first_not_null_item(['','1','']))

1


In [12]:
def get_second_not_null_item(lsts):
    filtered_data = [x for x in lsts if x != '']
    
    if len(filtered_data) >= 2:
        return filtered_data[1]
    else:
        return None
print(get_second_not_null_item(['','1', '7','']))

7


In [7]:
def get_detail_project(page, url,error_url_file,producer_info=[]):
    browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
    try:
        print("crawl url: ")
        print(url)
        browser.get(url)
        # sleep(2)
        # t = title.text
        wait = WebDriverWait(browser, 10)
        title = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "basicsSection-title desktop t-h5--sansSerif"))))
        description = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "basicsSection-tagline desktop t-body--sansSerif-lg"))))
        pledged = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "basicsGoalProgress-amountSold t-h5--sansSerif t-weight--bold"))))
        span_goal = browser.find_elements(By.CSS_SELECTOR, "basicsGoalProgress-progressDetails-detailsGoal-goalPercentageOrInitiallyRaised")
        percentage, value = span_goal.split(" of ")
        goal = value.strip()
        num_of_backer = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "t-weight--medium"))))
        days_to_go = get_second_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "t-weight--medium"))))
        field = list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR,"linkedLabelsList-tag-text"
        )))
        location = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "basicsCampaignOwner-details-city"))))
        
        num_of_comment = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR,
            "tabHeadersWithPill-tab-pill t-label--sm"
        ))))
        res = {
            "title":title,
            "description":description,
            "pledged":pledged,
            "goal":goal,
            "num_of_backer":num_of_backer,
            "days_to_go":days_to_go,
            "field":field,
            "location":location,
            "num_of_comment":num_of_comment
        }
        if (len(producer_info)>0):
            broker,topic = producer_info
            print("[*] Sending to broker "+broker+", topic: "+topic)
            print(json.dumps(res).encode("utf-8"))
            projectProducer = ProjectProducer(broker=broker)
            try:
                projectProducer.send_msg_sync(json.dumps(res).encode("utf-8"), topic=topic)
                print("[*] Send to kafka successfully.")
            except:
                print("[*] Send to kafka fail.") 
        else:
        #  file = open("./data/result.txt","a")
        #  file.write(url+"\n")
        #  file.write(str(res)+"\n")
        #  file.close()
          print("[*] Save data to mongodb.")
    except:
        error_url = str(page)+","+url+"\n"
        error_url_file_obj = open(error_url_file, "a")
        error_url_file_obj.write(error_url)
        error_url_file_obj.close()
        traceback.print_exc()
    browser.close()
# get_data(current_page=current_page)
get_detail_project(0,"https://www.kickstarter.com/projects/mlspencer/dragon-mage-deluxe-collectors-edition-hardcover",error_url_file,producer_info=[])

NameError: name 'error_url_file' is not defined

In [ ]:
broker,topic = "localhost:9092","kickstarter_project"
print("[*] Sending to broker "+broker+", topic: "+topic)
msg = "Oh my gosh."
print(msg)
projectProducer = ProjectProducer(broker=broker)
try:
    projectProducer.send_msg_sync(msg, topic=topic)
    print("[*] Send to kafka successfully.")
except:
    print("[*] Send to kafka fail.") 

In [ ]:
def get_data(url,current_page,num_of_thread,error_url_file,checkpoint_file):
    """
    This func is used to crawl data from Kickstarter website (
        https://www.kickstarter.com/discover/advanced?woe_id=0&sort=magic&seed=2811224&page=
    )

    Args:
        url (string): link to website
        current_page (int): current page index of page that is being crawled
        num_of_thread (int): num of crawling thread 
        error_url_file (string): name of file that contains a list of error_url_file that can not be crawled
        checkpoint_file (string): name of file that contains information about the index of page that is being crawled
    """
    page = current_page
    while (1):
        browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
        meta_url = url+str(page)
        print("meta_url: ")
        print(meta_url)
        try:
            browser.get(meta_url)
            sleep(2)
            links = list(map(lambda a: a.get_attribute("href"),
                                 browser.find_elements(By.CSS_SELECTOR,
                                                       "a[class='block img-placeholder w100p']")
                                 ))
            prj_links = [l for l in links if l.endswith("?ref=discovery")]
            print("prj_links: ")
            for l in prj_links:
                print(l)
            print(len(prj_links))
            threads = []
            split_prj_links = split_list(prj_links, num_of_thread)
            last_prj_links = split_prj_links[-1]
            print("split_prj_links: ")
            print(split_prj_links)
            for links in split_prj_links[:-1]:
                threads = [threading.Thread(
                        target=get_detail_project, args=(current_page, link,error_url_file)) for link in links]
                for thread in threads:
                    thread.start()
                for thread in threads:
                    thread.join()
                threads = []
            threads = [threading.Thread(
                    target=get_detail_project, args=(current_page, link,error_url_file)) for link in last_prj_links]
            for thread in threads:
                thread.start()
            for thread in threads:
                thread.join()
            threads = []
            page = page+1
        except:
            file = open(checkpoint_file, "w")
            file.write(str({"page": page}))
            traceback.print_exc()
            break
        browser.close()

In [ ]:
# link to the website Kickstarter
url = 'https://www.indiegogo.com/explore/all?project_type=campaign&project_timing=all&sort=trending'

# get the current page
checkpoint = eval(open('./data/checkpoint.csv', "r").readline())
current_page = checkpoint["page"]
error_url_file = 'error_url.csv'
print(current_page)

# start to crawl
app = FastAPI()
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)
@app.get("/projects")
def get_data_from_kickstarter(num_of_thread:int = 4):
 get_data(current_page=current_page,url=url,num_of_thread=num_of_thread,checkpoint_file="./data/checkpoint.csv",error_url_file="./da